<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/35_PyTorch_On_The_GPU_Training_Neural_Networks_With_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run PyTorch Code On A GPU - Neural Network Programming Guide

在本节中，我们将学习如何在PyTorch中使用GPU。 我们将看到一般如何使用GPU，还将看到如何将这些常规技术应用于训练我们的神经网络。


## 1.使用GPU进行深度学习

现在，我们将以PyTorch GPU示例为基础

### PyTorch GPU示例
--- 
在我们在程序内部执行计算时，PyTorch允许我们无缝地在GPU之间来回移动数据。

当我们使用GPU时，可以使用cuda（）方法，当我们使用CPU时，可以使用cpu（）方法。

我们也可以使用to（）方法。 要转到GPU，我们要写入to（'cuda'），要转到CPU，我们要写入to（'cpu'）。 to（）方法是首选方法，主要是因为它更灵活。 我们将看到使用前两个示例的示例，然后默认使用始终使用to（）变体的示例。

|CPU|GPU|
|:---:|:---:|
|cpu()|cuda()|
|to('cpu')|to('cuda')|

要在培训过程中使用我们的GPU，有两个基本要求。 这些要求如下:
* 数据必须移至GPU
* 网络必须移至GPU。

默认情况下，创建PyTorch张量或PyTorch神经网络模块时，将在CPU上初始化相应的数据。 具体来说，数据存在于CPU的内存中。

现在，让我们创建一个张量和一个网络，并看看我们如何进行从CPU到GPU的迁移。

在这里，我们创建一个张量和一个网络：



In [ ]:
# 实际的源码
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)

    self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
    self.fc2 = nn.Linear(in_features=120,out_features=60)
    self.out = nn.Linear(in_features=60,out_features=10)

  
  def forward(self,t):
    t= t

    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t,kernel_size=2,stride=2)

    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t,kernel_size=2,stride=2)

    t = t.reshape(-1,12*4*4)
    t = self.fc1(t)
    t = F.relu(t)

    t = self.fc2(t)
    t = F.relu(t)

    t = self.out(t)

    return t;


In [ ]:
t = torch.ones(1,1,28,28)
network = Network()

现在，我们调用cuda（）方法，并将张量和网络重新分配给已复制到GPU上的返回值：

In [ ]:
t =t.cuda()
network = network.cuda()

接下来，我们可以从网络获得预测，并看到预测张量的device属性确认数据在cuda（即GPU）上：

In [ ]:
gpu_pred = network(t)
gpu_pred.device

device(type='cuda', index=0)

同样，我们可以采取相反的方式：

In [ ]:
t = t.cpu()
network = network.cpu()

cpu_pred = network(t)
cpu_pred.device

device(type='cpu')

简而言之，这就是我们如何利用PyTorch的GPU功能。 现在，我们要讨论的是一些重要的细节，这些细节潜伏在我们刚刚看到的代码的表面之下。

例如，尽管我们使用了cuda（）和cpu（）方法，但实际上它们并不是我们的最佳选择。 此外，网络实例和张量实例之间的方法有什么区别？ 这些毕竟是不同的对象类型，这意味着这两种方法是不同的。 最后，我们希望将此代码集成到一个有效的示例中并进行性能测试。

### 使用GPU的总体思路
---

此时的主要结论是，我们的网络和数据都必须同时存在于GPU上，才能使用GPU进行计算，这适用于任何编程语言或框架。

我们将在下一个演示中看到，CPU也是如此。gpu和cpu是对数据进行计算的计算设备，因此在计算中直接相互使用的任何两个值必须存在于同一设备上。

## 2.基于GPU的PyTorch张量计算
让我们通过演示一些张量计算来更深入地研究。

我们将从创建两个张量开始：

In [ ]:
t1 = torch.tensor([
    [1,2],
    [3,4]
])

t2 = torch.tensor([
    [5,6],
    [7,8]
])

现在，我们将通过检查device属性来检查这些张量在哪个设备上初始化：

In [ ]:
t.device,t2.device

(device(type='cpu'), device(type='cpu'))

正如我们所期望的，我们看到，实际上，两个张量都在同一设备上，即CPU。 让我们将第一个张量t1移至GPU。

In [ ]:
t1 = t1.to('cuda')
t1.device

device(type='cuda', index=0)

我们可以看到，这个张量的设备已经改成了cuda，GPU。注意这里to（）方法的用法。我们不调用特定的方法来移动到设备，而是调用相同的方法并传递指定设备的参数。使用to（）方法是在设备之间移动数据的首选方法。

另外，请注意重新分配。操作不到位，因此需要重新分配。

让我们做个实验。我想通过对这两个张量t1和t2进行计算来测试我们之前讨论的内容，我们现在知道这两个张量在不同的设备上。

因为我们预期会出现错误，所以我们将调用包装为try并捕获异常：

In [ ]:
try:
  t1+t2
except Exception as e:
  print(e)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


通过颠倒操作顺序，我们可以看到错误也发生了变化：

In [ ]:
try:
  t2+t1
except Exception as e:
  print(e)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


这两个错误都告诉我们，二进制加号运算符期望第二个参数与第一个参数具有相同的设备。 调试这些类型的设备不匹配时，了解此错误的含义可能会有所帮助。

最后，为了完成操作，让我们将第二张量移动到cuda设备以查看操作是否成功。

In [ ]:
t2 = t2.to('cuda')
t1+t2

tensor([[ 6,  8],
        [10, 12]], device='cuda:0')

## 3.基于GPU的Pytorch nn.Module的计算

我们已经看到了如何在设备之间来回移动张量。 现在，让我们看看如何使用PyTorch nn.Module实例完成此操作。

更笼统地说，我们有兴趣了解网络在诸如GPU或CPU的设备上的含义以及含义。 撇开PyTorch，这是必不可少的问题。

通过将网络参数移至该设备，将网络放置在该设备上。 让我们创建一个网络，看看我们的意思。

In [ ]:
network = Network()

现在，让我们看一下网络的参数：

In [ ]:
for name,param in network.named_parameters():
  print(name,'\t\t',param.shape)

conv1.weight 		 torch.Size([6, 1, 5, 5])
conv1.bias 		 torch.Size([6])
conv2.weight 		 torch.Size([12, 6, 5, 5])
conv2.bias 		 torch.Size([12])
fc1.weight 		 torch.Size([120, 192])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([60, 120])
fc2.bias 		 torch.Size([60])
out.weight 		 torch.Size([10, 60])
out.bias 		 torch.Size([10])


在这里，我们创建了一个PyTorch网络，并迭代了该网络的参数。 如我们所见，网络的参数是网络内部的权重和偏差。

换句话说，这些只是存在于我们已经看到的设备上的张量。 让我们通过检查每个参数的设备来验证这一点。

In [ ]:
for n ,p in network.named_parameters():
  print(p.device,'',n)

cpu  conv1.weight
cpu  conv1.bias
cpu  conv2.weight
cpu  conv2.bias
cpu  fc1.weight
cpu  fc1.bias
cpu  fc2.weight
cpu  fc2.bias
cpu  out.weight
cpu  out.bias


这表明，默认情况下，网络内部的所有参数都是在CPU上初始化的。

一个重要的考虑因素是它解释了为什么像网络这样的nn.Module实例实际上没有设备。 设备上的网络不是网络，而是设备上的网络内部的张量。

让我们看看当我们要求将网络移动到GPU时会发生什么：

In [ ]:
network.to('cuda')

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

请注意，此处不需要重新分配实例来接收。 这是因为就网络实例而言，该操作是就地的。 所以，该操作可以用作重新分配操作。 对于nn.Module实例和PyTorch张量之间的一致性，首选此方法。

在这里，我们可以看到，现在所有网络参数都有一个cuda设备：

In [ ]:
for n ,p in network.named_parameters():
  print(p.device,'',n)

cuda:0  conv1.weight
cuda:0  conv1.bias
cuda:0  conv2.weight
cuda:0  conv2.bias
cuda:0  fc1.weight
cuda:0  fc1.bias
cuda:0  fc2.weight
cuda:0  fc2.bias
cuda:0  out.weight
cuda:0  out.bias


### 将样本传递到网络
---
让我们通过将示例传递到网络来结束本演示

In [ ]:
sample = torch.ones(1,1,28,28)
sample.shape

torch.Size([1, 1, 28, 28])

In [ ]:
try:
    network(sample)
except Exception as e: 
    print(e)

Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same


由于我们的网络位于GPU上，并且默认情况下此新创建的示例位于CPU上，因此我们会收到错误消息。 该错误告诉我们，在调用第一卷积层的forward方法时，CPU张量应为GPU张量。 这正是我们之前直接添加两个张量时所看到的。

我们可以通过将示例发送到GPU来解决此问题，如下所示：



In [ ]:
try:
    pred = network(sample.to('cuda'))
    print(pred)
except Exception as e:
    print(e)


tensor([[-0.1230,  0.1012, -0.0821, -0.1597,  0.1082, -0.0302,  0.0479,  0.0897,
          0.0565,  0.0540]], device='cuda:0', grad_fn=<AddmmBackward>)


最后，一切都按预期进行，我们得到了一个预测。

### 编写与设备无关的PyTorch代码
---
在总结之前，我们需要讨论编写与设备无关的代码。 这个与设备无关的术语意味着我们的代码不依赖于底层设备。 阅读PyTorch文档时，您可能会遇到此术语。

例如，假设我们编写的代码到处都使用cuda（）方法，然后将代码提供给没有GPU的用户。 这行不通。 不用担心 我们有选择！

还记得我们之前看到的cuda（）和cpu（）方法吗？

我们之所以首选to（）方法的原因之一是，因为to（）方法是参数化的，这使得更改我们选择的设备变得更加容易，即它很灵活！

例如，用户可以将cpu或cuda作为参数传递给深度学习程序，这将使该程序与设备无关。

允许程序用户传递确定程序行为的参数，这可能是使程序与设备无关的最佳方法。 但是，我们也可以使用PyTorch来检查受支持的GPU，并以此方式设置设备。

In [ ]:
torch.cuda.is_available()

True

就像，如果cuda可用，请使用它！

## 4.PyTorch GPU训练性能测试

现在让我们看看如何将GPU的使用添加到训练循环中。 我们将使用本系列到目前为止开发的代码进行此添加。

这将使我们能够轻松比较CPU和GPU的时间。

### 重构RunManager类
---

在更新训练循环之前，我们需要更新RunManager类。 在begin_run（）方法内部，我们需要修改传递给add_graph方法的图像张量的设备。

它看起来应该像这样：



```
def begin_run(self, run, network, loader):

    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.loader = loader
    self.tb = SummaryWriter(comment=f'-{run}')

    images, labels = next(iter(self.loader))
    grid = torchvision.utils.make_grid(images)

    self.tb.add_image('images', grid)
    self.tb.add_graph(
            self.network
        ,images.to(getattr(run, 'device', 'cpu'))
    )
```



在这里，我们使用内置的getattr（）函数来获取运行对象上设备的值。 如果运行对象没有设备，则返回cpu。 这使代码向后兼容。 如果我们没有为运行指定设备，它将仍然有效。

请注意，网络不需要移动到设备，因为它是在传入设备之前设置的。但是，图像张量是从加载程序获得的。


### 重构训练循环
---
我们将配置参数设置为具有设备。 这里的两个逻辑选项是cuda和cpu。



```
params = OrderedDict(
    lr = [.01]
    ,batch_size = [1000, 10000, 20000]
    , num_workers = [0, 1]
    , device = ['cuda', 'cpu']
)
```
将这些设备值添加到我们的配置中后，现在就可以在我们的训练循环中对其进行访问。

在run的顶部，我们将创建一个将在run内和训练循环内传递的设备。



```
device = torch.device(run.device)
```

我们将首先使用此设备的是在初始化网络时。


```
network = Network().to(device)
```

这将确保将网络移动到适当的设备。 最后，我们将通过分别解压缩图像和标签张量并将其发送到设备来更新图像和标签张量：


```
network = Network().to(device)
```
images = batch[0].to(device)
labels = batch[1].to(device)

这就是全部，我们已经准备好运行此代码并查看结果。

In [ ]:
# 完整代码
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)

    self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
    self.fc2 = nn.Linear(in_features=120,out_features=60)
    self.out = nn.Linear(in_features=60,out_features=10)

  
  def forward(self,t):
    t= t

    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t,kernel_size=2,stride=2)

    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t,kernel_size=2,stride=2)

    t = t.reshape(-1,12*4*4)
    t = self.fc1(t)
    t = F.relu(t)

    t = self.fc2(t)
    t = F.relu(t)

    t = self.out(t)

    return t;


In [ ]:
train_set = torchvision.datasets.FashionMNIST(root='./data',train=True,
                                download=True,
                                transform=transforms.Compose([
        transforms.ToTensor()
    ]))


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [ ]:
class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
          runs.append(Run(*v))
        return runs

In [ ]:
params = OrderedDict(
    lr = [0.01,0.001,0.0001],
    batch_size = [100,1000,10000,20000],
    num_workers=[0,1,2,4],
    device = ['cuda','cpu']
)

In [ ]:
for run in RunBuilder.get_runs(params):
  print(run.lr,run.batch_size,run.num_workers,run.device)

0.01 100 0 cuda
0.01 100 0 cpu
0.01 100 1 cuda
0.01 100 1 cpu
0.01 100 2 cuda
0.01 100 2 cpu
0.01 100 4 cuda
0.01 100 4 cpu
0.01 1000 0 cuda
0.01 1000 0 cpu
0.01 1000 1 cuda
0.01 1000 1 cpu
0.01 1000 2 cuda
0.01 1000 2 cpu
0.01 1000 4 cuda
0.01 1000 4 cpu
0.01 10000 0 cuda
0.01 10000 0 cpu
0.01 10000 1 cuda
0.01 10000 1 cpu
0.01 10000 2 cuda
0.01 10000 2 cpu
0.01 10000 4 cuda
0.01 10000 4 cpu
0.01 20000 0 cuda
0.01 20000 0 cpu
0.01 20000 1 cuda
0.01 20000 1 cpu
0.01 20000 2 cuda
0.01 20000 2 cpu
0.01 20000 4 cuda
0.01 20000 4 cpu
0.001 100 0 cuda
0.001 100 0 cpu
0.001 100 1 cuda
0.001 100 1 cpu
0.001 100 2 cuda
0.001 100 2 cpu
0.001 100 4 cuda
0.001 100 4 cpu
0.001 1000 0 cuda
0.001 1000 0 cpu
0.001 1000 1 cuda
0.001 1000 1 cpu
0.001 1000 2 cuda
0.001 1000 2 cpu
0.001 1000 4 cuda
0.001 1000 4 cpu
0.001 10000 0 cuda
0.001 10000 0 cpu
0.001 10000 1 cuda
0.001 10000 1 cpu
0.001 10000 2 cuda
0.001 10000 2 cpu
0.001 10000 4 cuda
0.001 10000 4 cpu
0.001 20000 0 cuda
0.001 20000 0 cpu
0.001 2

In [ ]:
# 理解上述思想，构建RunManager()类
class RunManager():
  def __init__(self):
    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    self.epoch_start_time = None
    self.epoch_num_workers=0


    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    self.network = None
    self.loader = None
    self.tb = None

  
  def begin_run(self,run,network,loader):
    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.loader = loader
    self.tb = SummaryWriter(comment=f'-{run}')

    images, labels = next(iter(self.loader))
    grid = torchvision.utils.make_grid(images)

    self.tb.add_image('images', grid)
    self.tb.add_graph(
            self.network
        ,images.to(getattr(run, 'device', 'cpu'))
    )

  def end_run(self):
    self.tb.close()
    self.epoch_count = 0

  def begin_epoch(self):
    self.epoch_start_time = time.time()
    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_num_correct = 0


  def end_epoch(self):

    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    loss = self.epoch_loss / len(self.loader.dataset)
    accuracy = self.epoch_num_correct / len(self.loader.dataset)

    self.tb.add_scalar('Loss', loss, self.epoch_count)
    self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

    for name, param in self.network.named_parameters():
        self.tb.add_histogram(name, param, self.epoch_count)
        self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)

    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results['loss'] = loss
    results["accuracy"] = accuracy
    results['epoch duration'] = epoch_duration
    results['run duration'] = run_duration
    results['num_workers'] = self.epoch_num_workers
    results['device']=self.run_params.device
    for k,v in self.run_params._asdict().items(): results[k] = v
    self.run_data.append(results)

    df = pd.DataFrame.from_dict(self.run_data, orient='columns')
    
    clear_output(wait=True)
    
    display(df)

  def get_num_workers(self,num_workers):
    self.epoch_num_workers = num_workers


  def track_loss(self, loss, batch):
    self.epoch_loss += loss.item() * batch[0].shape[0]

  def track_num_correct(self, preds, labels):
    self.epoch_num_correct += self.get_num_correct(preds, labels)

  def get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

  def save(self, fileName):

    pd.DataFrame.from_dict(
        self.run_data, orient='columns'
    ).to_csv(f'{fileName}.csv')

    with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
        json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [ ]:
runManger = RunManager()
for run in RunBuilder.get_runs(params):
  print(run)
  network = Network()
  network.to(run.device)
  train_loader = DataLoader(
      train_set,
      batch_size =run.batch_size,
      num_workers=run.num_workers
  )
  optimizer = optim.Adam(network.parameters(),lr=run.lr)
  runManger.begin_run(run,network,train_loader)
  for epoch in range(1):
    runManger.begin_epoch()
    runManger.get_num_workers(run.num_workers)
    
    for batch in train_loader:
      images,labels = batch
      images = images.to(run.device)
      labels = labels.to(run.device)      
      preds = network(images)

      loss = F.cross_entropy(preds,labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      runManger.track_loss(loss,batch)
      runManger.track_num_correct(preds,labels)
    runManger.end_epoch()
  runManger.end_run()








,run,epoch,loss,accuracy,epoch duration,run duration,num_workers,device,lr,batch_size
0,1,1,0.559973,0.788717,5.143286,5.242760,0,cuda,0.0100,100
1,2,1,0.572739,0.783117,19.920521,20.072048,0,cpu,0.0100,100
2,3,1,0.567751,0.785667,6.399574,6.592551,1,cuda,0.0100,100
3,4,1,0.563203,0.785683,19.446070,19.677835,1,cpu,0.0100,100
4,5,1,0.599108,0.772417,5.923637,6.164980,2,cuda,0.0100,100
...,...,...,...,...,...,...,...,...,...,...
66,67,1,1.489562,0.493483,6.389271,6.580999,1,cuda,0.0001,100
67,68,1,1.387307,0.506817,12.479754,12.710398,1,cpu,0.0001,100
68,69,1,1.309176,0.566733,5.956735,6.205655,2,cuda,0.0001,100
69,70,1,1.359743,0.522933,12.427641,12.725702,2,cpu,0.0001,100


Run(lr=0.0001, batch_size=100, num_workers=4, device='cpu')


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


KeyboardInterrupt: ignored

在这里，我们可以看到cuda设备的性能比cpu高出2倍至3倍。 结果可能会有所不同。